# Movie Recommender

In [1]:
import numpy as np
import pandas as pd

import pyspark as ps

In [2]:
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("mov_rec") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
rat_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/ratings.csv', header=True)
        

In [4]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [5]:
rat_rdd.tail(5)

[Row(userId=610, movieId=166534, rating=4.0, timestamp=1493848402),
 Row(userId=610, movieId=168248, rating=5.0, timestamp=1493850091),
 Row(userId=610, movieId=168250, rating=5.0, timestamp=1494273047),
 Row(userId=610, movieId=168252, rating=5.0, timestamp=1493846352),
 Row(userId=610, movieId=170875, rating=3.0, timestamp=1493846415)]

In [6]:
rat_rdd.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
mov_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/movies.csv', header=True)
        

In [8]:
# tags_rdd = spark.read.format('com.databricks.spark.csv').\
#                              options(header='true',\
#                              inferschema='true').\
#                              load('data/ml-latest-small/tags.csv', header=True)
        

In [9]:
mov_rdd.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [10]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [11]:
mov_rdd.createOrReplaceTempView("mov_rdd")
rat_rdd.createOrReplaceTempView("rat_rdd")

In [12]:
# tags_rdd.show(5)

Leave out timestamp...

In [13]:
mov_surprise = spark.sql(
'''
SELECT ratings.userId, movies.title, ratings.rating 
  FROM mov_rdd AS movies
  JOIN rat_rdd AS ratings
    ON movies.movieId = ratings.movieId
'''
)

In [14]:
mov_surprise.createOrReplaceTempView("mov_surprise")

In [15]:
mov_surprise.show(5)

+------+--------------------+------+
|userId|               title|rating|
+------+--------------------+------+
|     1|    Toy Story (1995)|   4.0|
|     1|Grumpier Old Men ...|   4.0|
|     1|         Heat (1995)|   4.0|
|     1|Seven (a.k.a. Se7...|   5.0|
|     1|Usual Suspects, T...|   5.0|
+------+--------------------+------+
only showing top 5 rows



# Similarity Based Recommender using Surprise

Begin by reimplementing the similarity based recommender.

In [16]:
# # install surprise if not already available
# pip install surprise

In [17]:
from surprise import KNNBasic, KNNWithMeans, SVD

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate

# mov_surprise doesn't contain any zero values
mov_surprise = mov_surprise[mov_surprise['rating']!=0]

# ratings in dataset are from 0.5 to 5
reader = Reader(rating_scale=(0.5, 5))

In [18]:
mov_surprise.head(5)

[Row(userId=1, title='Toy Story (1995)', rating=4.0),
 Row(userId=1, title='Grumpier Old Men (1995)', rating=4.0),
 Row(userId=1, title='Heat (1995)', rating=4.0),
 Row(userId=1, title='Seven (a.k.a. Se7en) (1995)', rating=5.0),
 Row(userId=1, title='Usual Suspects, The (1995)', rating=5.0)]

In [19]:
# convert to pandas DataFrame
# warning, if file is too large, this will likely crash...
mov_pDF = mov_surprise.toPandas()
print(mov_pDF.head(5))

surprise_movie_ratings = Dataset.load_from_df(mov_pDF, reader)

   userId                        title  rating
0       1             Toy Story (1995)     4.0
1       1      Grumpier Old Men (1995)     4.0
2       1                  Heat (1995)     4.0
3       1  Seven (a.k.a. Se7en) (1995)     5.0
4       1   Usual Suspects, The (1995)     5.0


In [20]:
print(mov_pDF.tail(5))

        userId                           title  rating
100831     610                    Split (2017)     4.0
100832     610   John Wick: Chapter Two (2017)     5.0
100833     610                  Get Out (2017)     5.0
100834     610                    Logan (2017)     5.0
100835     610  The Fate of the Furious (2017)     3.0


In [21]:
# treat data as training set
training_movie_data = surprise_movie_ratings.build_full_trainset()

# anything not a user-item rating pair as test
# no data for this, so we can't do an eval on out of sample accuracy
testing_movie_data = training_movie_data.build_anti_testset()

# create instance of recommender algorithm object
# algo = KNNBasic()
algo = SVD(n_factors=5, reg_all=0.005)

# fit it on the training data
algo.fit(training_movie_data)

# predict on the test data
predictions = algo.test(testing_movie_data)

In [22]:
# print the first 5 predictions
predictions[:5]

[Prediction(uid=1, iid='Shawshank Redemption, The (1994)', r_ui=3.501556983616962, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid='Good Will Hunting (1997)', r_ui=3.501556983616962, est=4.890305116848227, details={'was_impossible': False}),
 Prediction(uid=1, iid='Kill Bill: Vol. 1 (2003)', r_ui=3.501556983616962, est=4.883702987857899, details={'was_impossible': False}),
 Prediction(uid=1, iid='Collateral (2004)', r_ui=3.501556983616962, est=4.557143778577908, details={'was_impossible': False}),
 Prediction(uid=1, iid='Talladega Nights: The Ballad of Ricky Bobby (2006)', r_ui=3.501556983616962, est=4.2363893164188156, details={'was_impossible': False})]

In [23]:
# print last 5 predictions
predictions[-5:]

[Prediction(uid=610, iid='United States of Leland, The (2003)', r_ui=3.501556983616962, est=3.6221040598391436, details={'was_impossible': False}),
 Prediction(uid=610, iid='A Home at the End of the World (2004)', r_ui=3.501556983616962, est=3.667586599406033, details={'was_impossible': False}),
 Prediction(uid=610, iid='League of Ordinary Gentlemen, A (2004)', r_ui=3.501556983616962, est=3.6751790555013755, details={'was_impossible': False}),
 Prediction(uid=610, iid='I Think I Love My Wife (2007)', r_ui=3.501556983616962, est=3.464943951393655, details={'was_impossible': False}),
 Prediction(uid=610, iid='Man of the Year (1995)', r_ui=3.501556983616962, est=3.55851779347202, details={'was_impossible': False})]

## Cross Validate: SVD

In [24]:
# standard format for cross validation
# algorithm used, ratings dataset, metrics, number of cross validations, verbose controls how much output
cross_validate(SVD(n_factors=5, reg_all=0.005), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8809  0.8719  0.8666  0.8692  0.8690  0.8715  0.0050  
MAE (testset)     0.6757  0.6719  0.6657  0.6680  0.6679  0.6699  0.0035  
Fit time          3.16    2.96    3.27    3.24    3.19    3.17    0.11    
Test time         0.32    0.35    0.38    0.28    0.34    0.33    0.03    


{'test_rmse': array([0.88088657, 0.87186035, 0.8665543 , 0.86917713, 0.86901062]),
 'test_mae': array([0.67572328, 0.67191502, 0.665747  , 0.66798314, 0.66792964]),
 'fit_time': (3.164062023162842,
  2.9582197666168213,
  3.2725110054016113,
  3.243584156036377,
  3.1930036544799805),
 'test_time': (0.31595849990844727,
  0.34677624702453613,
  0.38074278831481934,
  0.2831912040710449,
  0.33979368209838867)}

## Cross Validtate: KNNBasic

In [25]:
cross_validate(KNNBasic(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9540  0.9642  0.9676  0.9557  0.9499  0.9583  0.0066  
MAE (testset)     0.7279  0.7320  0.7374  0.7294  0.7254  0.7304  0.0041  
Fit time          0.15    0.26    0.17    0.17    0.22    0.20    0.04    
Test time         2.48    2.57    2.22    2.28    2.50    2.41    0.14    


{'test_rmse': array([0.95398738, 0.96418061, 0.96756622, 0.95567385, 0.94990004]),
 'test_mae': array([0.72792781, 0.73204294, 0.73742989, 0.72942203, 0.72535068]),
 'fit_time': (0.1526050567626953,
  0.2585716247558594,
  0.1730971336364746,
  0.17201566696166992,
  0.22364521026611328),
 'test_time': (2.477376699447632,
  2.5741817951202393,
  2.224766492843628,
  2.275149345397949,
  2.5004022121429443)}

## Cross Validate: KNNWithMeans

In [26]:
cross_validate(KNNWithMeans(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9277  0.9343  0.9180  0.9370  0.9271  0.9288  0.0066  
MAE (testset)     0.7139  0.7172  0.7075  0.7212  0.7143  0.7148  0.0045  
Fit time          0.23    0.29    0.24    0.25    0.30    0.26    0.03    
Test time         2.53    2.64    2.74    2.79    2.56    2.65    0.10    


{'test_rmse': array([0.92765367, 0.93430686, 0.91799763, 0.93700528, 0.9270977 ]),
 'test_mae': array([0.71390581, 0.71722664, 0.70749369, 0.72121383, 0.71425625]),
 'fit_time': (0.22811603546142578,
  0.2907390594482422,
  0.24130463600158691,
  0.2465496063232422,
  0.30288243293762207),
 'test_time': (2.532083749771118,
  2.6400396823883057,
  2.7368369102478027,
  2.7885076999664307,
  2.5624494552612305)}

# Comparing to Baseline Model

In [27]:
from src import baselines 

# provide basline model our movie ratings
data = surprise_movie_ratings

print("\nGlobal Mean...")
algo_GM = baselines.GlobalMean()
cross_validate(algo_GM, data)

print("\nMeanOfMeans...")
algo_MM = baselines.MeanofMeans()
cross_validate(algo_MM, data)


Global Mean...

MeanOfMeans...


{'test_rmse': array([0.93325512, 0.93361289, 0.922333  , 0.93050177, 0.93444524]),
 'test_mae': array([0.73586904, 0.73285985, 0.72831928, 0.73104474, 0.7327901 ]),
 'fit_time': (1.1603779792785645,
  1.0722169876098633,
  1.1326982975006104,
  1.0795021057128906,
  1.0344197750091553),
 'test_time': (0.8386673927307129,
  0.8079900741577148,
  0.7589924335479736,
  0.7385973930358887,
  0.7042746543884277)}

In [28]:
import statistics

print(statistics.mean([0.92830858, 0.93106667, 0.9341514 , 0.93381002, 0.92575874]))

0.930619082


# Comparison of models

In [29]:
mom_rmse = 0.930619082
svd_rmse = 0.87

percent_change = abs(((svd_rmse - mom_rmse)) / abs(mom_rmse)) * 100
print(f"The new model shows a {percent_change:.2f}% improvement over the original")

The new model shows a 6.51% improvement over the original


## Search for Movie Title

## Top N Recommendations for each user

Source:
- [Top N Recommendations](https://surprise.readthedocs.io/en/stable/FAQ.html#top-n-recommendations-py)  

In [30]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# predictions has been instantiated above already
top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    print("\n")

1 ['Shawshank Redemption, The (1994)', 'Departed, The (2006)', 'Dark Knight, The (2008)', 'Philadelphia Story, The (1940)', "Singin' in the Rain (1952)", 'Rear Window (1954)', 'Casablanca (1942)', 'Brazil (1985)', 'Seven Samurai (Shichinin no samurai) (1954)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)']


2 ['Godfather, The (1972)', 'Fight Club (1999)', 'Streetcar Named Desire, A (1951)', 'Pulp Fiction (1994)', 'Lawrence of Arabia (1962)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 'Cool Hand Luke (1967)', 'Philadelphia Story, The (1940)', 'Forrest Gump (1994)', 'Boondock Saints, The (2000)']


3 ['Godfather, The (1972)', 'Braveheart (1995)', 'Trainspotting (1996)', 'Shawshank Redemption, The (1994)', 'American History X (1998)', 'Philadelphia Story, The (1940)', 'Goodfellas (1990)', 'Boondock Saints, The (2000)', 'Office Space (1999)', 'Streetcar Named Desire, A (1951)']


4 ['Godfather, The (1972)', 'Dr. Strangelo

In [31]:
for uid, iid, true_r, est, _ in predictions[:5]:
        print(uid, iid, true_r, est)

1 Shawshank Redemption, The (1994) 3.501556983616962 5
1 Good Will Hunting (1997) 3.501556983616962 4.890305116848227
1 Kill Bill: Vol. 1 (2003) 3.501556983616962 4.883702987857899
1 Collateral (2004) 3.501556983616962 4.557143778577908
1 Talladega Nights: The Ballad of Ricky Bobby (2006) 3.501556983616962 4.2363893164188156


# Predict results for user & movie

In [32]:
mov_pDF['title'].unique()

array(['Toy Story (1995)', 'Grumpier Old Men (1995)', 'Heat (1995)', ...,
       'Hazard (2005)', 'Blair Witch (2016)', '31 (2016)'], dtype=object)

In [33]:
def movie_title_search(str):
    
    title_lst = list()
    
    for i in mov_pDF['title'].unique():
        
        if str in i:
            title_lst.append(i)
    
    return title_lst  

print(movie_title_search('Matrix'))

['Matrix, The (1999)', 'Matrix Reloaded, The (2003)', 'Matrix Revolutions, The (2003)']
